# Make BOW Matrix of TED Talks

In [1]:
import pandas as pd
import numpy as np
import textacy
import json
from tqdm import tqdm 

In [2]:
a = pd.read_csv("data/transcripts.csv")

In [25]:
# corpus=[]
# docs = a['transcript'].tolist()
# for doc in tqdm(docs):
#     corpus.append(textacy.Doc(content=doc.decode('utf-8'),lang=u'en'))

In [26]:
c = textacy.Corpus(lang=u'en',texts=[i.decode('utf-8') for i in a['transcript'].tolist()])

In [28]:
vectorizer = textacy.Vectorizer(tf_type='linear', apply_idf=False, idf_type='smooth',
                                norm='l2', min_df=2, max_df=0.95)
doc_term_matrix = vectorizer.fit_transform( (doc.to_terms_list(ngrams=1,
                                                               named_entities=True,
                                                               as_strings=True) for doc in c))
print(repr(doc_term_matrix))

<2467x32966 sparse matrix of type '<type 'numpy.float64'>'
	with 1092422 stored elements in Compressed Sparse Row format>


In [21]:
help(textacy.Vectorizer)

Help on class Vectorizer in module textacy.vsm.vectorizers:

class Vectorizer(__builtin__.object)
 |  Transform one or more tokenized documents into a sparse document-term matrix
 |  of shape (# docs, # unique terms), with flexibly weighted and normalized values.
 |  
 |  Stream a corpus with metadata from disk::
 |  
 |      >>> cw = textacy.datasets.CapitolWords()
 |      >>> text_stream, metadata_stream = textacy.io.split_records(
 |      ...     cw.records(limit=1000), 'text', itemwise=False)
 |      >>> corpus = textacy.Corpus('en', texts=text_stream, metadatas=metadata_stream)
 |      >>> corpus
 |      Corpus(1000 docs; 538172 tokens)
 |  
 |  Tokenize and vectorize the first 600 documents of this corpus::
 |  
 |      >>> tokenized_docs = (
 |      ...     doc.to_terms_list(ngrams=1, named_entities=True, as_strings=True)
 |      ...     for doc in corpus[:600])
 |      >>> vectorizer = Vectorizer(
 |      ...     apply_idf=True, norm='l2',
 |      ...     min_df=3, max_df=0.95)

In [29]:
tokenized_docs = (doc.to_terms_list(ngrams=1, named_entities=True, as_strings=True) for doc in corpus)
vectorizer = textacy.Vectorizer(apply_idf=False,min_df=3, max_df=0.95)
doc_term_matrix = vectorizer.fit_transform(tokenized_docs)
BOW = pd.DataFrame(doc_term_matrix.toarray())
BOW

,0,1,2,3,4,5,6,7,8,9,...,587,588,589,590,591,592,593,594,595,596
0,0,0,0,0,0,1,2,0,0,2,...,1,7,2,6,1,8,0,2,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,1,1,2,0,0,1,1,0,1
2,3,1,0,2,2,1,0,0,0,0,...,1,1,9,2,1,8,2,0,1,0
3,3,0,0,0,0,1,0,0,2,0,...,1,3,1,1,1,2,0,0,5,1
4,1,4,3,0,0,6,1,1,0,0,...,0,0,1,0,0,11,0,0,0,0
5,1,5,2,2,1,0,3,2,0,2,...,0,3,4,1,0,13,2,1,3,0
6,0,1,0,0,0,0,0,0,0,0,...,0,5,1,1,4,6,8,0,0,1
7,0,0,0,0,0,1,3,1,1,0,...,0,1,0,0,0,5,0,0,1,0
8,0,0,1,0,0,0,0,0,0,1,...,2,4,1,3,0,16,3,1,0,0
9,0,2,0,1,1,0,2,1,1,0,...,3,2,4,0,0,14,0,1,0,0


In [30]:
BOW.to_csv("data/BOW")

# TF-IDF

In [32]:
tokenized_docs = (doc.to_terms_list(ngrams=1, named_entities=True, as_strings=True) for doc in corpus)
vectorizer = textacy.Vectorizer(apply_idf=True, norm='l2',min_df=3, max_df=0.95)
doc_term_matrix = vectorizer.fit_transform(tokenized_docs)
BOW_tf = pd.DataFrame(doc_term_matrix.toarray())
BOW_tf

,0,1,2,3,4,5,6,7,8,9,...,587,588,589,590,591,592,593,594,595,596
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.019233,0.038466,0.000000,0.000000,0.042832,...,0.019233,0.091811,0.026232,0.094728,0.021416,0.104927,0.000000,0.038466,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036920,...,0.000000,0.022611,0.022611,0.054435,0.000000,0.000000,0.033156,0.033156,0.000000,0.041527
2,0.065163,0.019507,0.000000,0.048863,0.048863,0.019507,0.000000,0.000000,0.000000,0.000000,...,0.019507,0.013303,0.119725,0.032026,0.021721,0.106422,0.039014,0.000000,0.021721,0.000000
3,0.057436,0.000000,0.000000,0.000000,0.000000,0.017193,0.000000,0.000000,0.043068,0.000000,...,0.017193,0.035175,0.011725,0.014114,0.019145,0.023450,0.000000,0.000000,0.095726,0.021534
4,0.016197,0.058183,0.054653,0.000000,0.000000,0.087274,0.014546,0.016197,0.000000,0.000000,...,0.000000,0.000000,0.009919,0.000000,0.000000,0.109114,0.000000,0.000000,0.000000,0.000000
5,0.017214,0.077294,0.038723,0.038723,0.019361,0.000000,0.046376,0.034427,0.000000,0.034427,...,0.000000,0.031627,0.042169,0.012690,0.000000,0.137048,0.030918,0.015459,0.051641,0.000000
6,0.000000,0.014940,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.050940,0.010188,0.012264,0.066542,0.061128,0.119516,0.000000,0.000000,0.018711
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.015977,0.047932,0.017791,0.020011,0.000000,...,0.000000,0.010896,0.000000,0.000000,0.000000,0.054479,0.000000,0.000000,0.017791,0.000000
8,0.000000,0.000000,0.019674,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017491,...,0.031416,0.042849,0.010712,0.038684,0.000000,0.171396,0.047125,0.015708,0.000000,0.000000
9,0.000000,0.032645,0.000000,0.020443,0.020443,0.000000,0.032645,0.018175,0.020443,0.000000,...,0.048968,0.022262,0.044525,0.000000,0.000000,0.155837,0.000000,0.016323,0.000000,0.000000


In [ ]:
BOW_tf.to_csv("data/tf-idf")